## Evaluate Agent Performance

In [ ]:
%pip install --quiet --upgrade pip==23.3.1

%pip install --upgrade --user --quiet "google-cloud-aiplatform[agent_engines,evaluation,langchain]" \
    "google-cloud-aiplatform" \
    "google-cloud-logging" \
    "google-cloud-aiplatform[autologging]" \
    "langchain_google_vertexai" \
    "cloudpickle==3.0.0" \
    "pydantic>=2.10" \
    "requests==2.32.3"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.0/101.0 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.8/103.8 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.2/148.2 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.9/17.9 MB 68.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [21]:
# General
import random
import string
import google.cloud.logging
import logging

from IPython.display import HTML, Markdown, display
import pandas as pd

# Build agent
import vertexai
from google.cloud import aiplatform
from vertexai import agent_engines

# Evaluate agent
from vertexai.preview.evaluation import EvalTask
from vertexai.preview.evaluation.metrics import (
    PointwiseMetric,
    PointwiseMetricPromptTemplate,
    TrajectorySingleToolUse,
)
from vertexai.preview.reasoning_engines import LangchainAgent

# Do not remove logging section
client = google.cloud.logging.Client()
client.setup_logging()

In [22]:
PROJECT_ID ="qwiklabs-gcp-01-9675d746eade"
LOCATION ="us-central1"
BUCKET_URI ="gs://qwiklabs-gcp-01-9675d746eade-experiments-staging-bucket"
EXPERIMENT_NAME ="evaluate-agent"

import vertexai
vertexai.init(project=PROJECT_ID, location=LOCATION,experiment=EXPERIMENT_NAME,staging_bucket=BUCKET_URI)

 ### Build agent and execute a query

In [23]:
# General
import random
import string

from IPython.display import HTML, Markdown, display

# Build agent
import vertexai
from google.cloud import aiplatform
from vertexai import agent_engines

from vertexai.preview.reasoning_engines import LangchainAgent

In [24]:
def get_product_details(product_name: str):
    """Gathers basic details about a product."""
    details = {
        "mens blue shorts": "Elevate your summer style with these tailored blue dress shorts. Featuring a modern slim fit and breathable cotton-blend fabric, they deliver all-day comfort with a refined look. Finished with a flat front, belt loops, and sleek pockets, they're perfect for everything from rooftop brunches to casual office days.",
        "floral dress": "Stay cool and stylish in this lightweight floral midi dress, featuring a flattering cinched waist, flowing A-line skirt, and delicate ruffle details. Perfect for sunny brunches or sunset strolls, it blends feminine charm with effortless ease for any summer occasion.",
        "garden furniture": "Create your perfect outdoor retreat with this stylish, weather-resistant garden furniture set featuring plush cushions, timeless design, and all-day comfort — ideal for relaxing, entertaining, or enjoying the outdoors in style.",
        "oled tv": "Experience stunning 4K clarity, vibrant color, and perfect blacks with this OLED Smart TV — featuring AI-powered optimization, cinematic sound, and a sleek design for immersive, next-gen home entertainment.",
        "dishwasher": "The SmartWash Dishwasher delivers powerful, quiet cleaning with advanced spray tech, smart cycles, and app control—saving time, water, and energy while adding modern style to your kitchen..",
    }
    return details.get(product_name, "Product details not found.")


def get_product_price(product_name: str):
    """Gathers price about a product."""
    details = {
        "mens blue shorts": 50,
        "floral dress": 100,
        "garden furniture": 1000,
        "oled tv": 1500,
        "dishwasher": 400,
    }
    return details.get(product_name, "Product price not found.")

In [25]:
model_name="gemini-2.0-flash"

In [26]:
local_1p_agent = LangchainAgent(
    model=model_name,
    tools=[get_product_details, get_product_price],
    agent_executor_kwargs={"return_intermediate_steps": True},
)

In [27]:
response = local_1p_agent.query(input="Get product details for garden furniture")
display(Markdown(response["output"]))

OK. I have the following details about garden furniture: "Create your perfect outdoor retreat with this stylish, weather-resistant garden furniture set featuring plush cushions, timeless design, and all-day comfort — ideal for relaxing, entertaining, or enjoying the outdoors in style."


In [28]:
response = local_1p_agent.query(input="Get product price for garden furniture")
display(Markdown(response["output"]))

The price of the garden furniture is 1000.


In [ ]:
remote_1p_agent = agent_engines.create(
    local_1p_agent,
    requirements=[
        "google-cloud-aiplatform[agent_engines,langchain]",
        "langchain_google_vertexai",
        "cloudpickle==3.0.0",
        "pydantic>=2.10",
        "requests==2.32.3",
    ],
)

INFO:vertexai.agent_engines:Identified the following requirements: {'pydantic': '2.11.7', 'cloudpickle': '3.0.0', 'google-cloud-aiplatform': '1.105.0'}
INFO:vertexai.agent_engines:The final list of requirements: ['google-cloud-aiplatform[agent_engines,langchain]', 'langchain_google_vertexai', 'cloudpickle==3.0.0', 'pydantic>=2.10', 'requests==2.32.3']
INFO:vertexai.agent_engines:Using bucket qwiklabs-gcp-01-9675d746eade-experiments-staging-bucket
INFO:vertexai.agent_engines:Wrote to gs://qwiklabs-gcp-01-9675d746eade-experiments-staging-bucket/agent_engine/agent_engine.pkl
INFO:vertexai.agent_engines:Writing to gs://qwiklabs-gcp-01-9675d746eade-experiments-staging-bucket/agent_engine/requirements.txt
INFO:vertexai.agent_engines:Creating in-memory tarfile of extra_packages
INFO:vertexai.agent_engines:Writing to gs://qwiklabs-gcp-01-9675d746eade-experiments-staging-bucket/agent_engine/dependencies.tar.gz
INFO:vertexai.agent_engines:Creating AgentEngine
INFO:vertexai.agent_engines:Create A

### Evaluate the agent using Single Tool Selection

In [29]:
eval_data = {
    "prompt": [
        "Get price for mens blue shorts",
        "Get product details and price for floral dress",
    ],
    "reference_trajectory": [
        [
            {
                "tool_name": "get_product_price",
                "tool_input": {"product_name": "mens blue shorts"},
            }
        ],
        [
            {
                "tool_name": "get_product_details",
                "tool_input": {"product_name": "floral dress"},
            },
            {
                "tool_name": "get_product_price",
                "tool_input": {"product_name": "floral dress"},
            },
        ],
    ],
}

eval_sample_dataset = pd.DataFrame(eval_data)

In [37]:
EXPERIMENT_RUN = "qwiklabs-gcp-01-9675d1746eade-single-tool-use"

single_tool_usage_metrics = [TrajectorySingleToolUse(tool_name="get_product_price")]

single_tool_call_eval_task = EvalTask(
    # Fill in the appropriate configuration
  dataset = eval_sample_dataset,
  metrics=single_tool_usage_metrics,

)


single_tool_call_eval_result = single_tool_call_eval_task.evaluate(
    runnable=remote_1p_agent,
     experiment_run_name=EXPERIMENT_RUN
)

100%|██████████| 2/2 [00:01<00:00,  1.69it/s]
INFO:vertexai.preview.evaluation._evaluation:All 2 responses are successfully generated from the runnable.
INFO:vertexai.preview.evaluation._evaluation:Computing metrics with a total of 2 Vertex Gen AI Evaluation Service API requests.
100%|██████████| 2/2 [00:00<00:00, 11.03it/s]
INFO:vertexai.preview.evaluation._evaluation:All 2 metric requests are successfully computed.
INFO:vertexai.preview.evaluation._evaluation:Evaluation Took:0.19616901700010203 seconds


### Evaluate the agent using Multiple Tool Selection (Trajectory) Evaluation

In [ ]:
trajectory_metrics = [
    "trajectory_exact_match",
    "trajectory_in_order_match",
    "trajectory_any_order_match",
    "trajectory_recall",
]

In [ ]:
EXPERIMENT_RUN = "qwiklabs-gcp-01-9675d746eade-agent-trajectory-evaluation"

trajectory_eval_task = EvalTask(
    # Fill in the appropriate configuration
    dataset = eval_sample_dataset,
  metrics=trajectory_metrics,
)

trajectory_eval_result = trajectory_eval_task.evaluate(runnable=remote_1p_agent,experiment_run_name=EXPERIMENT_RUN)

100%|██████████| 2/2 [00:01<00:00,  1.23it/s]
INFO:vertexai.preview.evaluation._evaluation:All 2 responses are successfully generated from the runnable.
INFO:vertexai.preview.evaluation._evaluation:Computing metrics with a total of 8 Vertex Gen AI Evaluation Service API requests.
100%|██████████| 8/8 [00:00<00:00, 10.35it/s]
INFO:vertexai.preview.evaluation._evaluation:All 8 metric requests are successfully computed.
INFO:vertexai.preview.evaluation._evaluation:Evaluation Took:0.7863749699999971 seconds


### Evaluate the quality of the agent response to prompts

In [31]:
response_metrics = ["safety", "coherence"]

In [33]:
EXPERIMENT_RUN = "qwiklabs-gcp-01-9675d746eade-agent-response-evaluation"

response_eval_task = EvalTask(
    # Fill in the appropriate configuration
     dataset = eval_sample_dataset,
    metrics=response_metrics
)

response_eval_result = response_eval_task.evaluate(runnable=remote_1p_agent,experiment_run_name=EXPERIMENT_RUN)

display(response_eval_result.metrics_table)


100%|██████████| 2/2 [00:01<00:00,  1.27it/s]
INFO:vertexai.preview.evaluation._evaluation:All 2 responses are successfully generated from the runnable.
INFO:vertexai.preview.evaluation._evaluation:Computing metrics with a total of 4 Vertex Gen AI Evaluation Service API requests.
100%|██████████| 4/4 [00:01<00:00,  3.48it/s]
INFO:vertexai.preview.evaluation._evaluation:All 4 metric requests are successfully computed.
INFO:vertexai.preview.evaluation._evaluation:Evaluation Took:1.1634858019999683 seconds


,prompt,reference_trajectory,response,latency_in_seconds,failure,predicted_trajectory,safety/explanation,safety/score,coherence/explanation,coherence/score
0,Get price for mens blue shorts,"[{'tool_name': 'get_product_price', 'tool_inpu...",The price for mens blue shorts is 50.,1.102159,0,"[{'tool_name': 'get_product_price', 'tool_inpu...",The response is safe because it does not conta...,1.0,The response is coherent as it provides a dire...,4.0
1,Get product details and price for floral dress,"[{'tool_name': 'get_product_details', 'tool_in...",Floral midi dress: Stay cool and stylish in t...,1.563149,0,"[{'tool_name': 'get_product_details', 'tool_in...",The response is safe as it does not contain an...,1.0,"The response is highly coherent, providing a c...",5.0


In [36]:

display(response_eval_result.metrics_table["coherence/explanation"][1])

'The response is highly coherent, providing a clear product description with relevant details and a price, creating a seamless and informative user experience.'